In [12]:
########## Loading Packages ##########
import numpy as np
import pandas as pd
from plotnine import *
from pandas.io.json import json_normalize
import json

In [135]:
########## Loading Ted Data ##########
ted = pd.read_csv('ted_full.csv')

In [136]:
########## Initializing New Columns ##########
##### ratings #####
for i in range(14):
    ted.loc[:, 'category_{}'.format(i)] = np.nan
for i in range(14):
    ted.loc[:, 'category_{}_count'.format(i)] = np.nan
    
##### related_talks #####
for i in range(6):
    ted.loc[:, 'related_speaker_{}'.format(i)] = np.nan
for i in range(6):
    ted.loc[:, 'related_title_{}'.format(i)] = np.nan
for i in range(6):
    ted.loc[:, 'related_slug_{}'.format(i)] = np.nan
for i in range(6):
    ted.loc[:, 'related_view_count_{}'.format(i)] = np.nan

In [140]:
########## Filling New Columns with JSON Data ##########
""" Taking JSON columns 'ratings', 'related_talks', and 'tags' and break them up into separate columns """

##### ratings #####
for i in range(ted.shape[0]):
    row = ted['ratings'][i]
    df = pd.DataFrame.from_dict(eval(row)).sort_values(by = 'count', ascending = False)
    ted.loc[i, 'category_0':'category_13_count'] = np.concatenate([df['name'].values, df['count'].values])
    
##### related_talks #####
for i in range(ted.shape[0]):
    row = ted['related_talks'][i]
    df = pd.DataFrame.from_dict(eval(row))
    shape = df.shape[0]
    if shape == 6:
        ted.loc[i, 'related_speaker_0':'related_view_count_5'] = np.concatenate(
            [df['speaker'].values, df['title'].values, df['slug'].values, df['viewed_count'].values])
    else:
        fill = np.empty(6 - shape)
        fill[:] = np.nan
        ted.loc[i, 'related_speaker_0':'related_view_count_5'] = np.concatenate(
            [df['speaker'].values, fill, 
             df['title'].values, fill,
             df['slug'].values, fill, 
             df['viewed_count'].values, fill])
        
##### tags #####
tags = ted['tags'].str.replace('[', '').str.replace(']', '').str.strip().str.split(expand = True)
tags.columns = ['tag_{}'.format(i) for i in range(38)]
ted = pd.concat([ted, tags], axis = 1)

In [234]:
########## Saving Data ##########
# ted.to_csv('ted_expanded.csv', index = False)